In [1]:
from tmgen.models import uniform_tm,spike_tm,modulated_gravity_tm,random_gravity_tm,gravity_tm,exp_tm
import csv
import os
import random

In [ ]:
class Work_load(object):
    def __init__(self,number_of_time_slots,file_result_path):
        self.T = []
        self.num_requests = 2
        
        self.each_t_requests = {0:[(1,6),(2,5),(10,12)],1:[(1,6),(2,5),(10,12)],2:[(1,6),(2,5),(10,12)]}
        self.each_t_real_requests = {1:[(1,6)],2:[(1,6)]}
        self.time_intervals = 2
        self.each_t_each_request_demand = {0:{(1,6):10,(2,5):0,(10,12):0},
                                           1:{(1,6):1,(2,5):0,(10,12):0},2:{(1,6):10,(2,5):0,(10,12):0}}
        self.each_request_threshold= {(1,6):{0:0.9,1:0.8,2:0.9},
                                     (2,5):{0:0.3,1:0.3,2:0.3},
                                     (10,12):{0:0.3,1:0.3,2:0.3}
                                     }
        self.each_user_request_fidelity_threshold = {(1,6):0.8,(2,10):0.9,(4,8):0.8,(6,11):0.9,
                                                          (5,10):0.8,(3,9):0.9,(8,10):0.8,(0,3):0.9,(1,10):1.0,(2,5):0.9}
        self.each_t_requests = {}
        self.each_t_real_requests = {}
        self.time_intervals = 2

        self.each_request_threshold= {}
        self.each_user_each_t_weight = {}
        
        for time in range(number_of_time_slots):
            if time not in self.T:
                self.T.append(time)
    def reset_variables(self):
        self.each_t_requests = {}
        self.each_t_real_requests = {}
    def set_each_user_weight_over_time(self,each_t_user_pairs):
        for t,user_pairs in each_t_user_pairs.items():
            for k in user_pairs:
                if t==0:
                    random_weight = random.uniform(0, 0.1)
                else:
                    random_weight = random.uniform(0, 1/t)
                
                try:
                    self.each_user_each_t_weight[k][t] = random_weight
                except:
                    self.each_user_each_t_weight[k]= {}
                    self.each_user_each_t_weight[k][t] = random_weight
    def get_each_user_weight_over_time(self,k,t):
        return self.each_user_each_t_weight[k][t]
    def set_each_time_requests(self,each_t_user_pairs,storage_pairs):
        for t,user_pairs in each_t_user_pairs.items():
            for pair in user_pairs:
                try:
                    if pair not in self.each_t_requests[t]:
                        try:
                            self.each_t_requests[t].append(pair)
                        except:
                            self.each_t_requests[t] = [pair]
                except:
                    self.each_t_requests[t] = [pair]
            for pair in storage_pairs:
                try:
                    if pair not in self.each_t_requests[t]:
                        try:
                            self.each_t_requests[t].append(pair)
                        except:
                            self.each_t_requests[t] = [pair]
                except:
                    self.each_t_requests[t] = [pair]
                    
    def set_each_time_real_requests(self,each_t_user_pairs):
        for t,user_pairs in each_t_user_pairs.items():
            for pair in user_pairs:
                try:
                    if pair not in self.each_t_real_requests[t]:
                        try:
                            self.each_t_real_requests[t].append(pair)
                        except:
                            self.each_t_real_requests[t] = [pair]
                except:
                    self.each_t_real_requests[t] = [pair]
    def set_each_testing_user_pair_demands(self,number_of_time_slots,each_t_user_pairs,spike_mean,num_spikes):
        self.each_t_each_request_demand = {}
        num_of_pairs= len(list(each_t_user_pairs[0]))
        #print(num_of_pairs,num_spikes,spike_mean,number_of_time_slots)
        tm = spike_tm(num_of_pairs+1,num_spikes,spike_mean,number_of_time_slots)
#         for time in range(number_of_time_slots):
#             traffic = tm.at_time(time)
#             printed_pairs = []
#             user_indx = 0
#             for i in range(num_of_pairs+1):
#                 for j in range(num_of_pairs+1):
#                     if i!=j:
#                         if (i,j) not in printed_pairs and (j,i) not in printed_pairs:
#                             printed_pairs.append((i,j))
#                             printed_pairs.append((j,i))
#                             #print("time %s traffic from %s to %s is %s"%(time,i,j,traffic[i][j]))
#                             request = each_t_user_pairs[time][user_indx]
#                             user_indx+=1
#                             demand = max(1,traffic[i][j])
#                             try:
#                                 self.each_t_each_request_demand[time][request] = demand
#                             except:
#                                 self.each_t_each_request_demand[time] = {}
#                                 self.each_t_each_request_demand[time][request] = demand
                                
        for time in range(0,number_of_time_slots):
            each_pair_demand = {}
            if time%2==0:
                for user_pair in each_t_user_pairs[time]:
                    demand  = random.randint(1, 1)
                    each_pair_demand[user_pair] = demand
            else:
                for user_pair in each_t_user_pairs[time]:
                    demand  = random.randint(1, 1)
                    each_pair_demand[user_pair] = demand
            if time==number_of_time_slots-1:
                for user_pair in each_t_user_pairs[time]:
                    demand  = random.randint(60, 60)
                    each_pair_demand[user_pair] = demand
#             each_pair_demand[user_pairs[0]] = demand1
#             each_pair_demand[user_pairs[1]] = demand2
            for request in each_t_user_pairs[time]:
                demand =each_pair_demand[request]
                try:
                    self.each_t_each_request_demand[time][request] = demand
                except:
                    self.each_t_each_request_demand[time] = {}
                    self.each_t_each_request_demand[time][request] = demand
                
                
        for request in each_t_user_pairs[time]:
            try:
                self.each_t_each_request_demand[0][request] = 0
            except:
                self.each_t_each_request_demand[0]={}
                self.each_t_each_request_demand[0][request] = 0
    def set_each_user_pair_demands_online(self,number_of_time_slots,given_each_t_each_pair_demands):
        #self.each_t_each_request_demand = {}
        for time,request_demand in given_each_t_each_pair_demands.items():
            for request,demand in request_demand.items():
                try:
                    self.each_t_each_request_demand[time][request] = given_each_t_each_pair_demands[time][request]
                except:
                    self.each_t_each_request_demand[time] = {}
                    self.each_t_each_request_demand[time][request] = given_each_t_each_pair_demands[time][request]
                                
            for request in given_each_t_each_pair_demands[time]:
                try:
                    self.each_t_each_request_demand[0][request] = 0
                except:
                    self.each_t_each_request_demand[0]={}
                    self.each_t_each_request_demand[0][request] = 0
    def set_each_user_pair_demands(self,number_of_time_slots,each_t_user_pairs,spike_mean,num_spikes):
        self.each_t_each_request_demand = {}
        num_of_pairs= len(list(each_t_user_pairs[0]))
#         print(num_of_pairs,num_spikes,spike_mean,number_of_time_slots)
#         print("each_t_user_pairs",each_t_user_pairs)
        tm = spike_tm(num_of_pairs+1,num_spikes,spike_mean,number_of_time_slots)
        for time in range(number_of_time_slots):
            traffic = tm.at_time(time)
            printed_pairs = []
            user_indx = 0
            for i in range(num_of_pairs):
                for j in range(num_of_pairs):
                    if i!=j:
                        if (i,j) not in printed_pairs and (j,i) not in printed_pairs and user_indx<num_of_pairs:
                            printed_pairs.append((i,j))
                            printed_pairs.append((j,i))
#                             print("num_of_pairs %s time %s traffic from %s to %s is %s and user_indx %s"%(num_of_pairs, time,i,j,traffic[i][j],user_indx))
                            request = each_t_user_pairs[time][user_indx]
                            user_indx+=1
                            demand = max(1,traffic[i][j])
                            try:
                                self.each_t_each_request_demand[time][request] = demand
                            except:
                                self.each_t_each_request_demand[time] = {}
                                self.each_t_each_request_demand[time][request] = demand
                                
#         for time in range(0,number_of_time_slots):
#             each_pair_demand = {}
#             if time%2==0:
#                 for user_pair in each_t_user_pairs[time]:
#                     demand  = random.randint(2, 3)
#                     each_pair_demand[user_pair] = demand
#             else:
#                 for user_pair in each_t_user_pairs[time]:
#                     demand  = random.randint(2, 3)
#                     each_pair_demand[user_pair] = demand
#             if time==number_of_time_slots-1:
#                 for user_pair in each_t_user_pairs[time]:
#                     demand  = random.randint(1000, 1500)
#                     each_pair_demand[user_pair] = demand
# #             each_pair_demand[user_pairs[0]] = demand1
# #             each_pair_demand[user_pairs[1]] = demand2
#             for request in each_t_user_pairs[time]:
#                 demand =each_pair_demand[request]
#                 try:
#                     self.each_t_each_request_demand[time][request] = demand
#                 except:
#                     self.each_t_each_request_demand[time] = {}
#                     self.each_t_each_request_demand[time][request] = demand
                
                
        for request in each_t_user_pairs[time]:
            try:
                self.each_t_each_request_demand[0][request] = 0
            except:
                self.each_t_each_request_demand[0]={}
                self.each_t_each_request_demand[0][request] = 0
#         with open(file_result_path, "r") as f:
#             reader = csv.reader(f, delimiter=",")
#             for line in (reader):
#                 time = int(line[0])
#                 request  = int(line[1])
#                 if time <15 and request <2:
                    
#                     request = each_user_id[request]
#                     demand = float(line[2])
                    
#                     if up_flag:
#                         demand = random.randint(2, 100)
#                         up_flag = False
#                     else:
#                         demand = random.randint(1, 3)
#                         up_flag  =True
#                     #print("demand is ",demand)
#                     try:
#                         self.each_t_each_request_demand[time][request] = demand
#                     except:
#                         self.each_t_each_request_demand[time] = {}
#                         self.each_t_each_request_demand[time][request] = demand
#                     if time not in self.T:
#                         self.T.append(time)
#                     try:
#                         if request not in self.each_t_requests[time]:
#                             self.each_t_requests[time].append(request)
#                     except:
#                         self.each_t_requests[time] = [request]

#                     if time >0:
#                         try:
#                             if self.each_t_real_requests[time]:
#                                 pass
#                         except:
#                             self.each_t_real_requests[time] = user_pairs
    def set_each_user_pair_demands_randomly(self,number_of_time_slots,each_t_user_pairs,spike_mean,num_spikes):
        self.each_t_each_request_demand = {}
        num_of_pairs= len(list(each_t_user_pairs[0]))
        
        for time in range(0,number_of_time_slots):
            each_pair_demand = {}
            if time%2==0:
                for user_pair in each_t_user_pairs[time]:
                    demand  = random.randint(1, spike_mean)
                    each_pair_demand[user_pair] = demand
            else:
                for user_pair in each_t_user_pairs[time]:
                    demand  = random.randint(1, spike_mean)
                    each_pair_demand[user_pair] = demand
            if time==number_of_time_slots-1:
                for user_pair in each_t_user_pairs[time]:
                    demand  = random.randint(1, spike_mean)
                    
                    each_pair_demand[user_pair] = demand
#             each_pair_demand[user_pairs[0]] = demand1
#             each_pair_demand[user_pairs[1]] = demand2
            for request in each_t_user_pairs[time]:
                demand =each_pair_demand[request]
                try:
                    self.each_t_each_request_demand[time][request] = demand
                except:
                    self.each_t_each_request_demand[time] = {}
                    self.each_t_each_request_demand[time][request] = demand
                
                
        for request in each_t_user_pairs[time]:
            try:
                self.each_t_each_request_demand[0][request] = 0
            except:
                self.each_t_each_request_demand[0]={}
                self.each_t_each_request_demand[0][request] = 0
    def get_each_t_whole_demands(self,time,user_pairs):
        sum_demands = 0
        for user_pair in user_pairs:
            sum_demands+= self.each_t_each_request_demand[time][user_pair]
        return sum_demands
    def check_demands_per_each_time(self,each_t_user_pairs):
        for time in self.T:
            for user_pair in each_t_user_pairs[time]:
                if self.each_t_each_request_demand[time][user_pair]==0:
                    self.each_t_each_request_demand[time][user_pair] = 1.0
            
    def set_storage_pairs_as_user_pairs(self, storage_pairs):
        for time in self.T:
            for pair in storage_pairs:
                try:
                    if pair not in self.each_t_requests[time]:
                        self.each_t_requests[time].append(pair)
                except:
                    self.each_t_requests[time] =[pair]
                try:
                    self.each_t_each_request_demand[time][pair] = 0
                except:
                    self.each_t_each_request_demand[time] = {}
                    self.each_t_each_request_demand[time][pair] = 0
    def set_threshold_fidelity_for_request_pairs(self,each_t_user_pairs,storage_pairs,each_user_request_fidelity_threshold):
        higest_threshold = []
        for time,pairs in each_t_user_pairs.items():
            for pair in pairs:
                try:
                    self.each_request_threshold[pair][time]= each_user_request_fidelity_threshold[pair]
                except:
                    self.each_request_threshold[pair] = {}
                    self.each_request_threshold[pair][time] = each_user_request_fidelity_threshold[pair] 
                higest_threshold.append(each_user_request_fidelity_threshold[pair])
        for pair in storage_pairs:
            for time in self.T:
                try:
                    self.each_request_threshold[pair][time] = 0.6
                except:
                    self.each_request_threshold[pair]= {}
                    self.each_request_threshold[pair][time] = 0.6
                    
#         print("higest_threshold",max(higest_threshold))
#         import time
#         time.sleep(5)
    def get_each_request_threshold(self,k,t):
        return self.each_request_threshold[k][t]
    def generate_workload_circle(self,alpha,T,request_pairs):
        new_t_request_demands = {}
        for t,request_demand in self.each_t_each_request_demand.items():
            new_t_request_demands[t] = {}
            for req,d in request_demand.items():
                new_t_request_demands[t][req] = d*alpha
        for k,v in new_t_request_demands.items():
            self.each_t_each_request_demand[k] = v
    

ZeroDivisionError: division by zero

In [ ]:
# def generate_demands(num_user_pairs, time_epochs):
#     each_minute_average_demand = {}
#     #for method in ["uniform","spike","exponential"]:
#     for method in ["spike"]:
#         each_minute_average_demand[method] = {}
#         each_minute_demands = {}
#         if method =="spike":
#             tm = spike_tm(num_user_pairs,2,100,time_epochs)
#         elif method =='uniform':
#             tm = uniform_tm(num_user_pairs,1,30,time_epochs)
#         elif method =="modulated_gravity":
#             tm = modulated_gravity_tm(num_user_pairs,time_epochs,100)
#         elif method == "random_gravity":
#             tm = random_gravity_tm(num_user_pairs,20,time_epochs)
#         elif method == "gravity":
#             tm = 1
#         elif method == "exponential":
#             tm = exp_tm(num_user_pairs,10,time_epochs)
#         print(tm)
#         for time in range(time_epochs):
#             traffic = tm.at_time(time)
#             print('for time %s we have traffic %s'%(time,traffic))
#             for demands in traffic:
#                 print('demands',demands)
#                 sum_demands= 0
#                 user_pair_indx = 0
#                 for demand in demands:
#                     print('demand',demand)
# #                     if not os.path.exists(time_demands_file):
# #                         os.mknod(time_demands_file)
#                     with open(time_demands_file, 'a') as newFile:                                
#                         newFileWriter = csv.writer(newFile)
#                         newFileWriter.writerow([user_pair_indx,time,demand]) 
#                     user_pair_indx+=1

#     return each_minute_average_demand
# time_demands_file = 'time_demands_file.csv'
# generate_demands(2,2)
# print("done!")

In [ ]:
# tm = spike_tm(3,2,100,2)
# print(tm)

In [ ]:
# #for time in [0,1]:
#     #traffic_time = tm.at_time(time) 
#     #print(traffic_time)
# time_demands_file = 'time_demands_file.csv'
# each_pair_src_dst = {(0,0):0,(0,1):1,(0,2):2,(0,3):3,(0,4):4,(0,5):5}
# pairs = [(0,0),(0,1),(0,2),(0,3),(0,4),(0,5)]
# tm = spike_tm(6,3,100,120)
# for pair in pairs:
#     traffic_over_times = tm.between(pair[0],pair[1],'all')
#     for time in range(120):
#         #print("at time %s from %s we have %s"%(time, each_pair_src_dst[pair],traffic_over_times[time]))
#         with open(time_demands_file, 'a') as newFile:                                
#             newFileWriter = csv.writer(newFile)
#             newFileWriter.writerow([time,each_pair_src_dst[pair],traffic_over_times[time]])

In [ ]:
# from tmgen.models import uniform_tm
# tm = uniform_tm(3, 100, 300)
# print(tm) 

In [ ]:
# from tmgen.models import exp_tm
# tm = exp_tm(3, 500, 2)

In [ ]:
# tm.matrix

In [ ]:
# tm.at_time(0) 

In [ ]:
class Network2:
    def __init__(self,topology_file_path,fidelity_experiment_flag,edge_fidelity_range,max_edge_capacity,fidelity_threshold_values):
        if fidelity_experiment_flag:
            self.fidelity_experiment = True
        else:
            self.fidelity_experiment = False
        self.set_E = []
        self.fidelity_threshold_values = fidelity_threshold_values
        self.max_edge_capacity_value = max_edge_capacity
        self.edge_fidelity_range = edge_fidelity_range
        self.each_edge_capacity = {(1,2):20,(2,3):20,(3,4):20,(4,5):20,(2,7):20,(7,8):20,(8,9):20,
                                   (9,5):20,(5,6):20,(2,10):20,(10,11):20,(11,12):20,(12,5):20}
        
        self.max_edge_capacity = 10
        #self.topology_file = data_dir + config.topology_file
        self.topology_file = topology_file_path
        self.set_E = [(1,2),(2,3),(3,4),(4,5),(2,7),(7,8),(8,9),(9,5),(5,6),(2,10),(10,11),(11,12),(12,5)]
        self.each_edge_capacity = {(1,2):20,(2,3):20,(3,4):20,(4,5):20,(2,7):20,(7,8):20,(8,9):20,
                                   (9,5):20,(5,6):20,(2,10):20,(10,11):20,(11,12):20,(12,5):20}
        
        self.max_edge_capacity = 10
        self.set_of_paths = {0:[(1,2),(2,10),(10,11),(11,12),(12,5),(5,6)],
                             1:[(2,3),(3,4),(4,5)],
                            2:[(2,7),(7,8),(8,9),(9,5)],
                             3:[(10,11),(11,12)],
                            4:[(1,2),(2,5),(5,6)],5:[(1,2),(2,5),(5,6)],
                            6:[(2,10),(10,12),(12,5)],
                            7:[(1,2),(2,10),(10,12),(12,5),(5,6)]}
        self.set_of_paths = {}
        self.each_path_basic_fidelity = {0:0.7,
                             1:0.8,
                            2:0.75,
                             3:0.9,
                            4:0.7,
                            5:0.6,
                            6:0.8,
                            7:0.8}
        self.oracle_for_target_fidelity={0:{0.7:2,0.8:3,0.9:2},
                             1:{0.7:2,0.8:3,0.9:2},
                            2:{0.7:3,0.8:4,0.9:2},
                             3:{0.7:3,0.8:4,0.9:2},
                            4:{0.7:3,0.8:4,0.9:2},
                            5:{0.7:3,0.8:4,0.9:2},
                            6:{0.7:3,0.8:4,0.9:2},
                            7:{0.7:2,0.8:4,0.9:2}} 
        self.each_request_real_paths = {(1,6):[0],(2,5):[1,2],(10,12):[3]}
        self.each_request_virtual_paths = {(1,6):[3,4,7],(2,5):[6],(10,12):[]}
        self.each_request_real_paths = {}
        self.each_request_virtual_paths = {}
        self.each_n_f_purification_result = {}
        self.storage_pairs = [(2,5),(10,12)]
        self.storage_nodes = [2,5,10,12]
        self.each_storage_capacity = {2:1000,5:1000,10:1000,12:1000}
        self.each_storage_capacity = {}
        self.each_edge_target_fidelity = {}
        self.storage_pairs = []
        self.storage_nodes = []
        self.each_user_pair_all_real_paths = {}
        self.each_user_pair_all_real_disjoint_paths = {}
        self.each_user_pair_real_paths = {}
        self.each_user_pair_virtual_paths = {}
        self.nodes = []
        self.each_t_user_pairs = {}
        self.each_storage_real_paths = {}
        self.each_request_virtual_paths_include_subpath = {}
        self.each_path_path_id = {}
        self.global_each_basic_fidelity_target_fidelity_required_EPRs = {}
        self.all_basic_fidelity_target_thresholds = []
        self.load_topology()
        
#         self.load_testing_topology()
#         self.g = nx.Graph()
#         self.modified_g = nx.Graph()
#         for each_edge in self.each_edge_capacity:
#             self.g.add_edge(each_edge[0],each_edge[1],weight=1)
#         self.g.add_edge(2,3,weight=1)
#         self.g.add_edge(3,4,weight=2)
#         self.g.add_edge(4,5,weight=1)
#         self.g.add_edge(5,6,weight=1)
#         self.g.add_edge(2,10,weight=1)
#         self.g.add_edge(10,11,weight=1)
#         self.g.add_edge(11,12,weight=1)
#         self.g.add_edge(12,5,weight=1)
#         self.g.add_edge(2,7,weight=1)
#         self.g.add_edge(7,8,weight=1)
#         self.g.add_edge(8,9,weight=1)
#         self.g.add_edge(9,5,weight=1)
        #nx.draw(g,with_labels=True)
        #plt.show()
        
        
        self.each_path_legth = {}
        
        #self.shortest_paths_file = self.topology_file +'_shortest_paths'
        #self.DG = nx.DiGraph()

        #self.load_topology()
        #self.calculate_paths()
        
    def load_topology(self):
       
        self.set_E=[]
        self.each_edge_capacity={}
        self.nodes = []
        self.each_edge_fidelity = {}
        self.max_edge_capacity = 0
        self.g = nx.Graph()
        print('[*] Loading topology...', self.topology_file)
        f = open(self.topology_file, 'r')
        header = f.readline()
        #f.readline()
#         self.link_capacities = np.empty((self.num_links))
#         self.link_weights = np.empty((self.num_links))
        for line in f:
            line = line.strip()
            link = line.split('\t')
            #print(line,link)
            i, s, d, c = link
            if int(s) not in self.nodes:
                self.nodes.append(int(s))
            if int(d) not in self.nodes:
                self.nodes.append(int(d))
            
            self.set_E.append((int(s),int(d)))
#             if self.fidelity_experiment:
#                 random_fidelity = random.uniform(0.95,0.98)
#             else:
            random_fidelity = random.uniform(self.edge_fidelity_range[0],self.edge_fidelity_range[1])
            self.each_edge_fidelity[(int(s),int(d))] = round(random_fidelity,3)
            self.each_edge_fidelity[(int(d),int(s))] = round(random_fidelity,3)
            random_capacity = random.randint(300, self.max_edge_capacity_value)
#             random_capacity = 200
            
            self.each_edge_capacity[(int(s),int(d))] = random_capacity
            if random_capacity>self.max_edge_capacity:
                self.max_edge_capacity  = random_capacity
            self.g.add_edge(int(s),int(d),weight=1)
#             self.link_capacities[int(i)] = float(c)
#             self.link_weights[int(i)] = int(w)
#             self.DG.add_weighted_edges_from([(int(s),int(d),int(w))])
#             self.set_E.append()
        
        f.close()
        #print('nodes: %d, links: %d\n'%(self.num_nodes, self.num_links))
#         nx.draw_networkx(self.DG)
#         plt.show()
        
    def load_testing_topology(self):
       
        self.set_E=[]
        self.each_edge_capacity={}
        self.nodes = []
        self.max_edge_capacity = 0
        self.g = nx.Graph()
        print('[*] Loading topology...', self.topology_file)
        f = open(self.topology_file, 'r')
        header = f.readline()
#         f.readline()
#         self.link_capacities = np.empty((self.num_links))
#         self.link_weights = np.empty((self.num_links))
        for line in f:
            line = line.strip()
            link = line.split('\t')
#             print(line,link)
            i, s, d, c = link
            if int(s) not in self.nodes:
                self.nodes.append(int(s))
            if int(d) not in self.nodes:
                self.nodes.append(int(d))
            
            self.set_E.append((int(s),int(d)))
            random_capacity = random.randint(200, 400)
            
            if (int(s),int(d)) ==(1,2) or (int(s),int(d))== (4,5) or (int(s),int(d)) == (2,1) or (int(s),int(d)) == (5,4):
                random_capacity = 100
            else:
                random_capacity = 10
#             print("for edge ",(int(s),int(d)),"capacity is ",random_capacity)
            self.each_edge_capacity[(int(s),int(d))] = random_capacity
            if random_capacity>self.max_edge_capacity:
                self.max_edge_capacity  = random_capacity
            self.g.add_edge(int(s),int(d),weight=1)

        
        f.close()  
        
    def get_user_pairs_over_dynamicly_chaning_population(self,number_of_user_pairs,distance_between_users,number_of_time_slots):
        self.each_t_user_pairs = {}
        candidate_user_pairs = []

        for src in self.nodes:
            for dst in self.nodes:
                if src!=dst and (src,dst) not in self.set_E and (dst,src) not in self.set_E:
                    shortest_path = nx.shortest_path(self.g, source=src, target=dst)
#                     if len(shortest_path)>=distance_between_users:
                    if (src,dst) not in candidate_user_pairs and (dst,src) not in candidate_user_pairs:
                        candidate_user_pairs.append((src,dst))
        #print("these are candidate pairs",candidate_user_pairs)
        #candidate_user_pairs = [(1,6),(2,10),(4,8),(6,11),(5,10),(3,9),(8,10),(0,3),(1,10),(2,5)]
        for t in range(number_of_time_slots):
            self.each_t_user_pairs[t] = []
        selected_user_pairs = []
        while(len(candidate_user_pairs)<number_of_time_slots *number_of_user_pairs):
            number_of_user_pairs =number_of_user_pairs-1
            
        for t in range(number_of_time_slots):
            while(len(self.each_t_user_pairs[t])<number_of_user_pairs):
                user_pair = candidate_user_pairs[random.randint(0,len(candidate_user_pairs)-1)]
                if user_pair not in self.each_t_user_pairs[t] and user_pair not in selected_user_pairs:
                    selected_user_pairs.append(user_pair)
                    try:
                        self.each_t_user_pairs[t].append(user_pair)
                    except:
                        self.each_t_user_pairs[t]=[user_pair]
    def get_testing_user_pairs(self,number_of_user_pairs,distance,number_of_time_slots):
        self.each_t_user_pairs = {}
        candidate_user_pairs = []
        for src in self.nodes:
            for dst in self.nodes:
                if src!=dst and (src,dst) not in self.set_E and (dst,src) not in self.set_E:
                    shortest_path = nx.shortest_path(self.g, source=src, target=dst)
                    if len(shortest_path)>=distance:
                        if (src,dst) not in candidate_user_pairs and (dst,src) not in candidate_user_pairs:
                            candidate_user_pairs.append((src,dst))
        candidate_user_pairs.append((1,5))
        #print("these are candidate pairs",candidate_user_pairs)
        #candidate_user_pairs = [(1,6),(2,10),(4,8),(6,11),(5,10),(3,9),(8,10),(0,3),(1,10),(2,5)]
        for t in range(number_of_time_slots):
            self.each_t_user_pairs[t] = []
        selected_fixed_user_pairs = []
        while(len(selected_fixed_user_pairs)<number_of_user_pairs):
            user_pair = candidate_user_pairs[random.randint(0,len(candidate_user_pairs)-1)]
            if user_pair not in selected_fixed_user_pairs:
                selected_fixed_user_pairs.append(user_pair)
        for t in range(number_of_time_slots):
            try:
                self.each_t_user_pairs[t]= [(1,5)]
            except:
                self.each_t_user_pairs[t]= [(1,5)]
    def get_user_pairs_online(self,number_of_user_pairs,distance_between_users,number_of_time_slots):
        each_t_user_pairs = {0: [(19, 24), (0, 18), (11, 23)], 
                     1: [(19, 24), (0, 18), (11, 23)], 
                     2: [(19, 24), (0, 18), (11, 23)], 
                     3: [(19, 24), (0, 18), (11, 23)], 
                     4: [(19, 24), (0, 18), (11, 23)], 
                     5: [(19, 24), (0, 18), (11, 23)], 
                     6: [(19, 24), (0, 18), (11, 23)], 
                     7: [(19, 24), (0, 18), (11, 23)], 
                     8: [(19, 24), (0, 18), (11, 23)], 
                     9: [(19, 24), (0, 18), (11, 23)], 
                     10: [(19, 24), (0, 18), (11, 23)], 
                     11: [(19, 24), (0, 18), (11, 23)], 
                     12: [(19, 24), (0, 18), (11, 23)], 
                     13: [(19, 24), (0, 18), (11, 23)], 
                     14: [(19, 24), (0, 18), (11, 23)]}
        for t,user_pairs in each_t_user_pairs.items():
            self.each_t_user_pairs[t]= user_pairs
        
    def get_user_pairs(self,number_of_user_pairs,distance,number_of_time_slots):
        #print("getting user pairs...")
        self.each_t_user_pairs = {}
        candidate_user_pairs = []
        while(len(candidate_user_pairs)<number_of_user_pairs):
            for src in self.nodes:
                for dst in self.nodes:
                    if src!=dst and (src,dst) not in self.set_E and (dst,src) not in self.set_E:
                        shortest_path = nx.shortest_path(self.g, source=src, target=dst)
                        #if len(shortest_path)>=distance:
                        if (src,dst) not in candidate_user_pairs and (dst,src) not in candidate_user_pairs:
                            candidate_user_pairs.append((src,dst))
        for t in range(number_of_time_slots):
            self.each_t_user_pairs[t] = []
        selected_fixed_user_pairs = []
        while(len(selected_fixed_user_pairs)<number_of_user_pairs):
            user_pair = candidate_user_pairs[random.randint(0,len(candidate_user_pairs)-1)]
            if user_pair not in selected_fixed_user_pairs:
                selected_fixed_user_pairs.append(user_pair)
        for pair in selected_fixed_user_pairs:
            self.each_id_pair[self.user_id] = pair
            self.each_pair_id[pair] = self.pair_id
            selected_ids.append(self.pair_id)
            self.pair_id+=1
        for t in range(number_of_time_slots):
            try:
                self.each_t_user_pairs[t]= selected_ids
            except:
                self.each_t_user_pairs[t]= selected_ids

                    
    def set_user_pair_fidelity_threshold(self,fidelity_threshold_range):
        self.each_user_request_fidelity_threshold = {}
        possible_thresholds = self.fidelity_threshold_values
        possible_thresholds_based_on_given_range = []
        
        possible_thresholds_based_on_given_range.append(fidelity_threshold_range)
        for t,user_pairs in self.each_t_user_pairs.items():
            for pair in user_pairs:
                self.each_user_request_fidelity_threshold[pair] = possible_thresholds_based_on_given_range[random.randint(0,len(possible_thresholds_based_on_given_range)-1)]

    def set_each_path_length(self,path_id,path):
        self.each_path_legth[path_id] = len(path)
    
    def check_if_request_uses_this_sub_path(self,k,p,p_s):
        if p in self.each_request_virtual_paths_include_subpath[k][p_s]:
            return True
        else:
            return False
        
    def remove_storage_pair_real_path_from_path(self,sub_path,path):
        A = path
        B = sub_path
        start_ind = None
        for i in range(len(A)):
            if A[i:i+len(B)] == B:
                start_ind = i
                break

        C = [x for i, x in enumerate(A) 
             if start_ind is None or not(start_ind <= i < (start_ind + len(B)))]
            
        return C
    def get_next_fidelity_and_succ_prob(self,F):
        succ_prob = (F+((1-F)/3))**2 + (2*(1-F)/3)**2
        output_fidelity = (F**2 + ((1-F)/3)**2)/succ_prob

        return output_fidelity, succ_prob

    def get_avg_epr_pairs(self,F_init,F_target):
        F_curr = F_init
        n_avg = 1.0
        while(F_curr < F_target):
            F_curr,succ_prob = self.get_next_fidelity_and_succ_prob(F_curr)
            n_avg = n_avg*(2/succ_prob)
        return  n_avg

    def get_avg_output_fidelity(self,F_init,n_avg):
        F_curr = F_init
        n_curr = 1
        while(1):
            F_prev,succ_prob = self.get_next_fidelity_and_succ_prob(F_curr)
            n_curr = n_curr*(2/succ_prob)

            if(n_curr > n_avg):
                break
            else:
                F_curr = F_prev

        return F_curr
    def recursive_purification(self,n,f):
        #print("n is ",n)
        n = int(n)
        if f ==1.0:
            return f
        else:
            if n<=0 or (n==1):
                return f
            else:
                numerator=  f * self.recursive_purification(n-1,f)
                denominator = numerator + (1-f) * (1-self.recursive_purification(n-1,f))

                return round(numerator/(denominator),4)
    
    def get_possible_threshold_for_each_n(self,basic_fidelities):
    #         for path in self.set_of_paths:
    #             for f in [0.6,0.7,0.8,0.9,1.0]:
    #                 try:
    #                     self.oracle_for_target_fidelity[path][f] = 4
    #                 except:
    #                     self.oracle_for_target_fidelity[path] = {}
    #                     self.oracle_for_target_fidelity[path][f] = 4
        n_values = []
        each_basic_fidelity_each_EPR_number_target_fidelity = {}

        for f in basic_fidelities:
            n = 1
            each_basic_fidelity_each_EPR_number_target_fidelity[f] = {}
            final_fidelity = f
            if n not in n_values:
                n_values.append(n)
            each_basic_fidelity_each_EPR_number_target_fidelity[f][n] = final_fidelity
            #print("for basic fidelity ",str(f))
            while n <20:
                n+=1
                if n not in n_values:
                    n_values.append(n)
                final_fidelity = self.recursive_purification(n,f)
                each_basic_fidelity_each_EPR_number_target_fidelity[f][n] = final_fidelity
            #print("for basic fidelity ",str(f),"we are done!")
        return each_basic_fidelity_each_EPR_number_target_fidelity,n_values
    
    def get_next_fidelity_and_succ_prob_BBPSSW(self,F):
        succ_prob = (F+((1-F)/3))**2 + (2*(1-F)/3)**2
        output_fidelity = (F**2 + ((1-F)/3)**2)/succ_prob

        return output_fidelity, succ_prob

    def get_next_fidelity_and_succ_prob_DEJMPS(self,F1,F2,F3,F4):
        succ_prob = (F1+F2)**2 + (F3+F4)**2
        output_fidelity1 = (F1**2 + F2**2)/succ_prob
        output_fidelity2 = (2*F3*F4)/succ_prob
        output_fidelity3 = (F3**2 + F4**2)/succ_prob
        output_fidelity4 = (2*F1*F2)/succ_prob

        return output_fidelity1, output_fidelity2, output_fidelity3, output_fidelity4, succ_prob

    def get_avg_epr_pairs_BBPSSW(self,F_init,F_target):
        F_curr = F_init
        n_avg = 1.0
        while(F_curr < F_target):
            F_curr,succ_prob = get_next_fidelity_and_succ_prob_BBPSSW(F_curr)
            n_avg = n_avg*(2/succ_prob)
        return  n_avg

    def get_avg_epr_pairs_DEJMPS(self,F_init,F_target):
        F_curr = F_init
        F2 = F3 = F4 = (1-F_curr)/3
        n_avg = 1.0
        while(F_curr < F_target):
            F_curr,F2, F3, F4, succ_prob = get_next_fidelity_and_succ_prob_DEJMPS(F_curr, F2, F3, F4)
            n_avg = n_avg*(2/succ_prob)
        return  n_avg
    
    
    def set_required_EPR_pairs_for_each_path_each_fidelity_threshold(self):
        targets = []
        for t in self.fidelity_threshold_values:
            targets.append(t)
        targets.append(0.6)
        targets.sort()
        for path,path_basic_fidelity in self.each_path_basic_fidelity.items():
            #print("for path %s with lenght %s fidelity %s"%(path,self.each_path_legth[path],path_basic_fidelity))
            try:
                if path_basic_fidelity in self.global_each_basic_fidelity_target_fidelity_required_EPRs:
                    
                    for target in targets:
                        
                        #print("getting required rounds for initial F %s to target %s path length %s"%(path_basic_fidelity,target,self.each_path_legth[path]))
                        n_avg = self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity][target]
                        #print("we got ",n_avg)
                        try:
                            self.oracle_for_target_fidelity[path][target] = n_avg
                            
                        except:
                            self.oracle_for_target_fidelity[path] = {}
                            self.oracle_for_target_fidelity[path][target] = n_avg
                            
                else:
                    
                    for target in targets:
                        
                        #print("getting required rounds for initial F %s to target %s path lenght %s "%(path_basic_fidelity,target,self.each_path_legth[path]))
                        n_avg = self.get_avg_epr_pairs(path_basic_fidelity ,target)
                        n_avg = self.get_avg_epr_pairs_DEJMPS(path_basic_fidelity ,target)
                        #print("we got ",n_avg)
                        try:
                            self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity][target] =n_avg 
                        except:
                            self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity]={}
                            self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity][target] =n_avg 
                            
                        try:
                            self.oracle_for_target_fidelity[path][target] = n_avg
                            
                        except:
                            self.oracle_for_target_fidelity[path] = {}
                            self.oracle_for_target_fidelity[path][target] = n_avg
                            
            except:
                
                for target in targets:
                    n_avg = self.get_avg_epr_pairs(path_basic_fidelity ,target)
                    try:
                        self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity][target] =n_avg 
                    except:
                        self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity]={}
                        self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity][target] =n_avg                     
                    try:
                        self.oracle_for_target_fidelity[path][target] = n_avg
                        
                    except:
                        self.oracle_for_target_fidelity[path] = {}
                        self.oracle_for_target_fidelity[path][target] = n_avg
                        
        
    def set_required_EPR_pairs_for_path_fidelity_threshold(self):
        targets = []
        for t in self.fidelity_threshold_values:
            targets.append(t)
        targets.append(0.6)
        targets.sort()
        basic_fidelities = []
        for path,basic_fidelity in self.each_path_basic_fidelity.items():
            if basic_fidelity not in self.all_basic_fidelity_target_thresholds:
                self.all_basic_fidelity_target_thresholds.append(basic_fidelity)
                if basic_fidelity not in basic_fidelities:
                    basic_fidelities.append(basic_fidelity)
                    
        if basic_fidelities:
#             print("we are going to get required EPRs to each thresholds for basics",basic_fidelities)
#             time.sleep(10)
            each_basic_fidelity_each_EPR_number_target_fidelity , n_values=self.get_possible_threshold_for_each_n(basic_fidelities)
            each_basic_fidelity_target_fidelity_required_EPRs = {}
            each_basic_available_targets= {}
            for basic_fidelity,n_final_fidelity in each_basic_fidelity_each_EPR_number_target_fidelity.items():
                each_basic_fidelity_target_fidelity_required_EPRs[basic_fidelity] = {}
                each_basic_available_targets[basic_fidelity] = {}
                available_targets = []
                for n,final_fidelity in n_final_fidelity.items():
            #         print("with basic %s to reach %s you need %s"%(basic_fidelity,final_fidelity,n))
                    if final_fidelity<=0.6:
                        target_fidelity_threshold = 0.6
                    elif final_fidelity<=0.65:
                        target_fidelity_threshold = 0.65
                    elif final_fidelity<=0.7:
                        target_fidelity_threshold = 0.7
                    elif final_fidelity<=0.75:
                        target_fidelity_threshold = 0.75
                    elif final_fidelity<=0.8:
                        target_fidelity_threshold = 0.8
                    elif final_fidelity<=0.85:
                        target_fidelity_threshold = 0.85
                    elif final_fidelity<=0.9:
                        target_fidelity_threshold = 0.9
                    elif final_fidelity<=0.95:
                        target_fidelity_threshold = 0.95 
                    elif final_fidelity<=0.98:
                        target_fidelity_threshold = 0.98
                    elif final_fidelity<=1:
                        target_fidelity_threshold = 1
            #         print("for basic fidelity %s with %s you will get %s "%(basic_fidelity,n,final_fidelity))
                    if target_fidelity_threshold not in available_targets:
                        available_targets.append(target_fidelity_threshold)
                    try:
                        each_basic_fidelity_target_fidelity_required_EPRs[basic_fidelity][target_fidelity_threshold].append(n)
                    except:
                        each_basic_fidelity_target_fidelity_required_EPRs[basic_fidelity][target_fidelity_threshold]=[n]
    #             print("available_targets        ",available_targets)
    #             print("targets                  ",targets)

                each_basic_available_targets[basic_fidelity] = available_targets
                targets.sort()
                for target in targets:
                    if target <= basic_fidelity:
                        each_basic_fidelity_target_fidelity_required_EPRs[basic_fidelity][target]=[1]
                        try:
                            if target not in each_basic_available_targets[basic_fidelity]:
                                each_basic_available_targets[basic_fidelity].append(target)
                        except:
                            each_basic_available_targets[basic_fidelity]=[target]
                for target in targets:
            #         print("with basic %s for target %s"%(basic_fidelity,target))
                    if target in available_targets:
                        last_target_threshold_n = min(each_basic_fidelity_target_fidelity_required_EPRs[basic_fidelity][target])+1
                        last_available_target_threshold = target
                    if target not in available_targets:
                        try:
                            each_basic_fidelity_target_fidelity_required_EPRs[basic_fidelity][target].append(last_target_threshold_n)
                        except:
                            each_basic_fidelity_target_fidelity_required_EPRs[basic_fidelity][target]=[last_target_threshold_n]



            for basic in each_basic_fidelity_target_fidelity_required_EPRs:
                for path,path_basic_fidelity in self.each_path_basic_fidelity.items():
                    if path_basic_fidelity==basic:
                        for target in targets:
                            required_EPRs = each_basic_fidelity_target_fidelity_required_EPRs[basic][target]
                            try:
                                self.oracle_for_target_fidelity[path][target] = min(required_EPRs)
#                                 print("for path %s to target %s we added %s"%(path,target,min(required_EPRs)))
                            except:
                                self.oracle_for_target_fidelity[path] = {}
                                self.oracle_for_target_fidelity[path][target] = min(required_EPRs)
#                                 print("for path %s to target %s we added %s"%(path,target,min(required_EPRs)))
                                
            for basic,target_EPRs in each_basic_fidelity_target_fidelity_required_EPRs.items():
                self.global_each_basic_fidelity_target_fidelity_required_EPRs[basic] = target_EPRs
                
            for path,path_basic_fidelity in self.each_path_basic_fidelity.items():
                if path_basic_fidelity not in basic_fidelities:
                    for target in targets:
                        required_EPRs = self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity][target]
                        try:
                            self.oracle_for_target_fidelity[path][target] = min(required_EPRs)
#                             print("from storage: for path %s to target %s we added %s"%(path,target,min(required_EPRs)))
                        except:
                            self.oracle_for_target_fidelity[path] = {}
                            self.oracle_for_target_fidelity[path][target] = min(required_EPRs)
#                             print("from storage: for path %s to target %s we added %s"%(path,target,min(required_EPRs)))
#             for path,target_n in self.oracle_for_target_fidelity.items():
#                 for t,n in target_n.items():
#                     print("for path %s to reach %s we need %s"%(path,t,n))
#             time.sleep(5)
        else:
#             print("we have already computed required EPR pairs to reach threshold for each basic fidelity")
#             time.sleep(10)
            for path,path_basic_fidelity in self.each_path_basic_fidelity.items():
                for target in targets:
                    required_EPRs = self.global_each_basic_fidelity_target_fidelity_required_EPRs[path_basic_fidelity][target]
                    try:
                        self.oracle_for_target_fidelity[path][target] = min(required_EPRs)
#                         print("from storage: for path %s to target %s we added %s"%(path,target,min(required_EPRs)))
                    except:
                        self.oracle_for_target_fidelity[path] = {}
                        self.oracle_for_target_fidelity[path][target] = min(required_EPRs)
                
#         for p,bf in self.each_path_basic_fidelity.items():
#             print("path basic fidelity",p,bf)
#         print(self.oracle_for_target_fidelity.keys())
#         for p,target_EPR in self.oracle_for_target_fidelity.items():
#             for target,EPR in target_EPR.items():
#                 print("for path %s with F %s to reach %s you need %s",p,self.each_path_basic_fidelity[p],target,EPR)
#         import time
#         time.sleep(2)
#         for path,target_EPRs in self.oracle_for_target_fidelity.items():
#             for target,EPRs in target_EPRs.items():
#                 print("for path %s for threshold %s we need %s"%(path,target,EPRs))
                        
#                         print("from storage: for path %s to target %s we added %s"%(path,target,min(required_EPRs)))
#             for path,target_n in self.oracle_for_target_fidelity.items():
#                 for t,n in target_n.items():
#                     print("for path %s to reach %s we need %s"%(path,t,n))
#             time.sleep(5)
            #import pdb
            #pdb.set_trace()
            
            
#         for path in self.set_of_paths:
#             for f in [0.6,0.7,0.8,0.9,1.0]:
#                 try:
#                     self.oracle_for_target_fidelity[path][f] = 4
#                 except:
#                     self.oracle_for_target_fidelity[path] = {}
#                     self.oracle_for_target_fidelity[path][f] = 4
#         for path in self.set_of_paths:
#             for f in [0.6,0.7,0.8,0.9,1.0]:
#                 n = 1
#                 final_fidelity = f
#                 if final_fidelity!=1.0:
#                     while final_fidelity <=0.999:
#                         n+=1
#                         final_fidelity = self.recursive_purification(n,f)
#                 else:
#                     n=1
#                 try:
#                     self.oracle_for_target_fidelity[path][f] = n
#                 except:
#                     self.oracle_for_target_fidelity[path] = {}
#                     self.oracle_for_target_fidelity[path][f] = n
    def set_storage_capacity(self):
        for storage_node in self.storage_nodes:
            self.each_storage_capacity[storage_node] =1000
    def get_required_edge_level_purification_EPR_pairs(self,edge,p,each_t_requests):
        longest_p_lenght   = 0
        max_F_threshold = 0
        for p,path in self.set_of_paths.items():
            if edge in path:
                if len(path)>longest_p_lenght:
                    longest_p_lenght = len(path)
                for user in each_t_requests:
                    print("self.each_user_pair_real_paths",self.each_user_pair_real_paths)
                    if p in self.each_user_pair_real_paths[user]+self.each_user_pair_virtual_paths[user]:
                        if self.each_user_request_fidelity_threshold[user] > max_F_threshold:
                            max_F_threshold = self.each_user_request_fidelity_threshold[user]
        required_edge_level_fidleity = (3*(4/3*max_F_threshold-1/3)**(1/longest_p_lenght)+1)/4
        new_target = required_edge_level_fidleity
        edge_basic_fidelity = self.each_edge_fidelity[edge]
        try:
            if new_target in self.each_edge_target_fidelity[edge]:
                return self.each_edge_target_fidelity[edge][new_target]
            else:
                n_avg = self.get_avg_epr_pairs_DEJMPS(edge_basic_fidelity ,new_target)
                try:
                    self.each_edge_target_fidelity[edge][new_target] = n_avg
                except:
                    self.each_edge_target_fidelity[edge] = {}
                    self.each_edge_target_fidelity[edge][new_target] = n_avg
                return n_avg
        except:
                required_edge_level_fidleity = (3*(4/3*max_F_threshold-1/3)**(1/longest_p_lenght)+1)/4
                new_target = required_edge_level_fidleity
                edge_basic_fidelity = self.each_edge_fidelity[edge]
                n_avg = self.get_avg_epr_pairs_DEJMPS(edge_basic_fidelity ,new_target)
                try:
                    self.each_edge_target_fidelity[edge][new_target] = n_avg
                except:
                    self.each_edge_target_fidelity[edge] ={}
                    self.each_edge_target_fidelity[edge][new_target] = n_avg
                return n_avg
    def get_required_purification_EPR_pairs(self,p,threshold):
#         print("we are getting the required EPR pairs for path %s to reach threshold %s"%(p,threshold))
        
        return self.oracle_for_target_fidelity[p][threshold]
        if threshold>=0.9:
            return self.oracle_for_target_fidelity[p][0.9]
        elif 0.8<threshold<0.9:
            return self.oracle_for_target_fidelity[p][0.8]
        elif threshold<0.8:
            return self.oracle_for_target_fidelity[p][0.7]
        else:
            return 1
    def get_testing_new_storage_pairs(self,number_of_storages): 
        if number_of_storages>0:
            self.storage_pairs.append((2,4))
            self.storage_nodes.append(2)
            self.storage_nodes.append(4)
        else:
            self.storage_pairs= []
            self.storage_nodes= []
    def reset_storage_pairs(self):
        self.storage_pairs=[]
        self.storage_nodes=[]
    def get_new_storage_pairs(self,number_of_storages,storage_node_selection_scheme):
        if storage_node_selection_scheme == "Degree":
            #print("we are selecting storage nodes")
            import time
            all_user_pairs = []
            #time.sleep(2)
            user_pair_nodes = set([])
            for t,user_pairs in self.each_t_user_pairs.items():
                for user_pair in user_pairs:
                    if user_pair not in all_user_pairs:
                        all_user_pairs.append(user_pair)
                    if (user_pair[1],user_pair[0]) not in all_user_pairs:
                        all_user_pairs.append((user_pair[1],user_pair[0]))
                    user_pair_nodes.add(user_pair[0])
                    user_pair_nodes.add(user_pair[1])
            user_pair_nodes = list(user_pair_nodes)
            each_degree_nodes = {}
            degrees = []
            candidate_nodes = []
            for node in self.g.nodes:
                #if node not in user_pair_nodes:
                    try:
                        each_degree_nodes[self.g.degree[node]].append(node)
                    except:
                        each_degree_nodes[self.g.degree[node]]= [node]
                    degrees.append(self.g.degree[node])
            degrees.sort(reverse=True)
            for degree in degrees:
                nodes = each_degree_nodes[degree]
                for n in nodes:
                    if n not in candidate_nodes:
                        candidate_nodes.append(n)
            #print("candidate_nodes",candidate_nodes)
            if len(candidate_nodes)==0:
                any_available_nodes_for_storage = False
            else:
                any_available_nodes_for_storage = True
            while(len(self.storage_nodes)<number_of_storages and len(candidate_nodes)>0 ):    
                    storage1 = candidate_nodes[0]
                    candidate_nodes.pop(0)
                    if self.storage_pairs:
                        for storage_node in self.storage_nodes:
                            if ((storage1,storage_node) not in self.storage_pairs and (storage_node,storage1) not in self.storage_pairs) and storage1!=storage_node:
                                if (storage1,storage_node) not in all_user_pairs:
                                    self.each_id_pair[self.pair_id] = (storage1,storage_node)
                                    self.each_pair_id[(storage1,storage_node)] = self.pair_id
                                    self.storage_pairs.append(self.pair_id)
                                    self.pair_id+=1
                                    
                        if storage1 not in self.storage_nodes:
                            if storage1 not in user_pair_nodes:
                                self.storage_nodes.append(storage1)
                    else:
                        if len(candidate_nodes)>1:
                            storage1 = candidate_nodes[0]
                            storage2 = candidate_nodes[1]
                            candidate_nodes.pop(0)
                            if (storage1,storage2) not in self.storage_pairs and (storage2,storage1) not in self.storage_pairs:
                                if (storage1,storage2) not in all_user_pairs:
                                    self.storage_pairs.append(self.pair_id)
                                    self.each_id_pair[self.pair_id] = (storage1,storage2)
                                    self.each_pair_id[(storage1,storage2)] = self.pair_id
                                    self.pair_id+=1
                                    
                                    self.storage_nodes.append(storage1)
                                    self.storage_nodes.append(storage2)
            
        else:
            new_selected_storage_nodes = []
            new_selected_storage_pairs = []
            user_pair_nodes = set([])
            all_user_pairs = []
            for t,user_pairs in self.each_t_user_pairs.items():
                for user_pair in user_pairs:
                    user_pair_nodes.add(user_pair[0])
                    user_pair_nodes.add(user_pair[1])
                    
                    if user_pair not in all_user_pairs:
                        all_user_pairs.append(user_pair)
                    if (user_pair[1],user_pair[0]) not in all_user_pairs:
                        all_user_pairs.append((user_pair[1],user_pair[0]))
            user_pair_nodes = list(user_pair_nodes)
            permitted_nodes = []
            for node in self.nodes:
                if node not in self.storage_nodes:
                    permitted_nodes.append(node)
            #print("we have permitted node as ",permitted_nodes,len(self.storage_nodes),number_of_storages,len(user_pair_nodes),user_pair_nodes)
            if len(permitted_nodes)+len(self.storage_nodes)<=number_of_storages:
                for storage1 in permitted_nodes:
                    for storage2 in permitted_nodes:
                        if storage1 != storage2:
                            if (storage1,storage2) not in self.storage_pairs and (storage2,storage1) not in self.storage_pairs:
                                if (storage1,storage2) not in all_user_pairs:
                                    self.storage_pairs.append(self.pair_id)
                                    self.each_id_pair[self.pair_id] = (storage1,storage2)
                                    self.each_pair_id[(storage1,storage2)] = self.pair_id
                                    self.pair_id+=1
                                    
                                    #self.storage_pairs.append((storage1,storage2))
                                    self.storage_nodes.append(storage1)
                                    self.storage_nodes.append(storage2)

            else:
                while(len(self.storage_nodes)<number_of_storages and len(permitted_nodes)>0):
                    storage1 = permitted_nodes[0]
                    while(storage1 in self.storage_nodes):
                        storage1 = permitted_nodes[random.randint(0,len(permitted_nodes)-1)]
                    #print("we selected new storage node %s"%(storage1))
                    if self.storage_pairs:
                        for storage_node in self.storage_nodes:
                            if ((storage1,storage_node) not in self.storage_pairs and (storage_node,storage1) not in self.storage_pairs) and storage1!=storage_node:
                                if (storage1,storage_node) not in all_user_pairs:
                                    self.storage_pairs.append(self.pair_id)
                                    self.each_id_pair[self.pair_id] = (storage1,storage_node)
                                    self.each_pair_id[(storage1,storage_node)] = self.pair_id
                                    self.pair_id+=1
                                    
                        if storage1 not in self.storage_nodes:
                            self.storage_nodes.append(storage1)
                    else:
                        storage1 = permitted_nodes[random.randint(0,len(permitted_nodes)-1)]
                        storage2 = storage1
                        while(storage1==storage2):
                            storage2 = permitted_nodes[random.randint(0,len(permitted_nodes)-1)]
                        #print("for round  we have new storage node %s"%(storage1))
                        if (storage1,storage2) not in self.storage_pairs and (storage2,storage1) not in self.storage_pairs:
                            if (storage1,storage2) not in all_user_pairs:
                                self.storage_pairs.append(self.pair_id)
                                self.each_id_pair[self.pair_id] = (storage1,storage2)
                                self.each_pair_id[(storage1,storage2)] = self.pair_id
                                self.pair_id+=1
                                
                                self.storage_nodes.append(storage1)
                                self.storage_nodes.append(storage2)
                        #print("this is our first storage pair",self.storage_pairs)
        #return any_available_nodes_for_storage
    def reset_pair_paths(self):
        self.set_of_paths = {}
        self.each_user_pair_all_real_paths = {}
        self.each_user_pair_all_real_disjoint_paths = {}
        self.each_request_real_paths = {}
        self.each_request_virtual_paths = {}
        self.each_request_virtual_paths_include_subpath = {}
        self.each_path_path_id = {}
        self.each_storage_real_paths = {}
    
    def get_each_user_pair_real_paths(self,pairs):
        #print("we are getting all paths for pairs ",pairs)
        for user_pair in pairs:
            shortest_paths = nx.all_shortest_paths(self.g,source=user_pair[0],target=user_pair[1], weight='weight')
            #print(shortest_paths)
            #print(shortest_paths.sort(reverse=True))
            import pdb
            #pdb.set_trace()
            paths = []
            for p in shortest_paths:
                #print("for src %s dst %s shortest path is %s "%(user_pair[0],user_pair[1],p))
                paths.append(p)
            self.each_user_pair_all_real_paths[user_pair] = paths
        for user_pair in pairs:
            if user_pair[0]==user_pair[1]:
                paths = [[user_pair[0]]]
            else:
                shortest_disjoint_paths = nx.edge_disjoint_paths(self.g,s=user_pair[0],t=user_pair[1])
                #print(shortest_paths)
                #print(shortest_paths.sort(reverse=True))
                import pdb
                #pdb.set_trace()
                paths = []
                for p in shortest_disjoint_paths:
                    #print("for src %s dst %s shortest disjoint path is %s "%(user_pair[0],user_pair[1],p))
                    #if p not in self.each_user_pair_all_real_paths[user_pair]:
                        #print("We have a path in disjoint that is not in shortest path!!!!")
                        #print("the shortespt paths are ",self.each_user_pair_all_real_paths[user_pair])
                        #time.sleep(4)
                    paths.append(p)
            self.each_user_pair_all_real_disjoint_paths[user_pair] = paths
        
        #time.sleep(1)
            
    def get_real_longest_path(self,user_or_storage_pair,number_of_paths):
        all_paths=[]
        for path in nx.all_simple_paths(self.g,source=user_or_storage_pair[0],target=user_or_storage_pair[1]):
            #all_paths.append(path)

            node_indx = 0
            path_edges = []
            for node_indx in range(len(path)-1):
                path_edges.append((path[node_indx],path[node_indx+1]))
                node_indx+=1
            all_paths.append(path_edges)

        all_paths.sort(key=len,reverse=True)
        if len(all_paths)>=number_of_paths:
            return all_paths[:number_of_paths]
        else:
            return all_paths
                        
    def get_real_path(self,user_or_storage_pair,number_of_paths,path_selection_scheme):
        if path_selection_scheme=="shortest":
            path_selecion_flag = False
            path_counter = 1
            paths = []
            #print("self.each_user_pair_all_real_paths[user_or_storage_pair]",self.each_user_pair_all_real_paths[user_or_storage_pair])
            for path in self.each_user_pair_all_real_paths[user_or_storage_pair]:
                #print("we can add this path",path)
                if path_counter<=number_of_paths:
                    node_indx = 0
                    path_edges = []
#                     print("for pair this is the normal path ",user_or_storage_pair,path)
#                     if len(path)==1:
#                         print("this si the unnormal case")
#                         for node_indx in range(len(path)-1):
#                             print("edge",(path[node_indx],path[node_indx+1]))
#                             path_edges.append((path[node_indx],path[node_indx+1]))
#                             node_indx+=1
#                         import pdb
#                         import time
#                         time.sleep(3)
#                         #pdb.set_trace()
                    for node_indx in range(len(path)-1):
                        path_edges.append((path[node_indx],path[node_indx+1]))
                        node_indx+=1
                    paths.append(path_edges)

                path_counter+=1
        elif path_selection_scheme=="shortest_disjoint":
            path_selecion_flag = False
            path_counter = 1
            paths = []
            #print("self.each_user_pair_all_real_paths[user_or_storage_pair]",self.each_user_pair_all_real_paths[user_or_storage_pair])
            for path in self.each_user_pair_all_real_disjoint_paths[user_or_storage_pair]:
                #print("we can add this path",path)
                if path_counter<=number_of_paths:
                    node_indx = 0
                    path_edges = []
#                     print("for pair this is the disjoint path ",user_or_storage_pair,path)
#                     if len(path)==1:
#                         print("this si the unnormal case",path)
#                         for node_indx in range(len(path)-1):
#                             print("edge",(path[node_indx],path[node_indx+1]))
#                             path_edges.append((path[node_indx],path[node_indx+1]))
#                             node_indx+=1
#                         import pdb
#                         #pdb.set_trace()
                    for node_indx in range(len(path)-1):
                        path_edges.append((path[node_indx],path[node_indx+1]))
                        node_indx+=1
                    paths.append(path_edges)

                path_counter+=1
            
        return paths
                    
      
                    
    def connect_users_to_storages(self,src,dst,str1,str2,real_sub_path,number_of_paths,path_selection_scheme):
        #print("we are going to connect node %s to %s and %s to %s"%(src,str1,str2,dst))
        self.get_each_user_pair_real_paths([(src,str1)])
        self.get_each_user_pair_real_paths([(str2,dst)])
        sub_paths1 = self.get_real_path((src,str1),number_of_paths,path_selection_scheme)
        sub_paths2 = self.get_real_path((str2,dst),number_of_paths,path_selection_scheme)
        #print("we got these paths for them",sub_paths1,sub_paths2)
        set_of_paths = []
        for path_part1 in sub_paths1:
            new_path = []
            for edge in path_part1:
                new_path.append(edge)
            for edge in real_sub_path:
                new_path.append(edge)
            back_up_path = []
            for e in new_path:
                back_up_path.append(e)
            for path_part2 in sub_paths2:
                for edge in path_part2:
                    new_path.append(edge)
                set_of_paths.append(new_path)
                new_path = back_up_path
        return set_of_paths
    def get_paths_to_connect_users_to_storage(self,user_pair,real_sub_path,num_paths,path_selection_scheme):
        #print("we are going to find shortest path to connect user pair %s to sub path %s"%(user_pair,real_sub_path))
        src= user_pair[0]
        dst= user_pair[1]
        str1 = real_sub_path[0][0]
        str2= real_sub_path[len(real_sub_path)-1][1]
        first_set_of_paths = self.connect_users_to_storages(src,str1,str2,dst,real_sub_path,num_paths,path_selection_scheme)
        second_set_of_paths = self.connect_users_to_storages(dst,str1,str2,src,real_sub_path,num_paths,path_selection_scheme)
#         print("we got first set of paths ",first_set_of_paths)
#         print("we got the second set of paths",second_set_of_paths)
        first_path_length = []
        second_path_length = []
        for path in first_set_of_paths:
            first_path_length.append(len(path))
        for path in second_set_of_paths:
            second_path_length.append(len(path))
        if sum(first_path_length)/len(first_path_length)>sum(second_path_length)/len(second_path_length):
            return second_set_of_paths
        else:
            return first_set_of_paths
        
    def set_each_path_basic_fidelity(self):
#         print("self.each_edge_fidelity[edge]",self.each_edge_fidelity)
        #basic_fidelities = [0.7,0.8,0.9]
        self.each_path_basic_fidelity = {}
        for path,path_edges in self.set_of_paths.items():
            #basic_fidelity = basic_fidelities[random.randint(0,len(basic_fidelities)-1)]
            
            basic_fidelity = 1/4+(3/4)*(4*self.each_edge_fidelity[path_edges[0]]-1)/3
            #1/4 +3/4((4F1-1)/3)((4F2-1)/3)
            #print("for path %s with lenght %s first edge fidelity %s %s "%(path,self.each_path_legth[path],self.each_edge_fidelity[path_edges[0]],basic_fidelity ))
            for edge in path_edges[1:]:
#                 basic_fidelity = basic_fidelity * self.each_edge_fidelity[edge]
                #print("edge with fidelity %s"%(self.each_edge_fidelity[edge]))
                basic_fidelity  = (basic_fidelity)*((4*self.each_edge_fidelity[edge]-1)/3)
            basic_fidelity = basic_fidelity
            self.each_path_basic_fidelity[path]= round(basic_fidelity,3)
            #if round(basic_fidelity,3)<0.6:
                #print("path end to end fidelity is ",round(basic_fidelity,3))
                #import time
                #time.sleep(10)
    def check_each_request_real_virtual_paths(self,T,each_t_requests):
        for t in T[1:]:
            for k in each_t_requests[t]:
                having_at_least_one_path_flag = False
                this_k_set_of_real_paths = []
                this_k_set_of_virtual_paths = []
                try:
                    if self.each_request_real_paths[k]:
                        for p in self.each_request_real_paths[k]:
                            having_at_least_one_path_flag = True
                except:
                    pass
                try:
                    for p in self.each_request_virtual_paths[k]:
                        having_at_least_one_path_flag = True
                except:
                    pass
                if not having_at_least_one_path_flag:
                    return False
        return having_at_least_one_path_flag
                    
    def check_path_include_sub_path2(self,k,sub_path_id,path_id):
        if path_id in self.each_request_virtual_paths_include_subpath[k][sub_path_id]:
            return True
        else:
            return False   
    def check_path_include_sub_path(self,sub_path,path):
        if self.set_of_paths[sub_path] in self.set_of_paths[path]:
            return True
        else:
            return False
    def get_edges(self):
        return self.set_E
    def get_this_path_fidelity(self,path_edges):
        
        
        basic_fidelity = 1/4+(3/4)*(4*self.each_edge_fidelity[path_edges[0]]-1)/3
        #1/4 +3/4((4F1-1)/3)((4F2-1)/3)
        for edge in path_edges[1:]:
            basic_fidelity  = (basic_fidelity)*((4*self.each_edge_fidelity[edge]-1)/3)
        return basic_fidelity
    def get_storage_capacity(self,storage):
        return self.each_storage_capacity[storage]
    def check_path_include_edge(self,edge,path):
        #print('edge is %s and path is %s and Paths is %s'%(edge,path,self.set_of_paths))
        if edge in self.set_of_paths[path]:
            return True
        elif edge not  in self.set_of_paths[path]:
            return False
    def check_storage_pair_exist(self,s1,s2):
        if (s1,s2) in self.storage_pairs:
            return True
        else:
            return False
    def check_request_use_path(self,k,p):
        if p in self.each_request_virtual_paths[k] or (p in self.each_request_virtual_paths[k]):
            return True
        else:
            return False
    #         edge_capacity
    #         paths
    #         virtual_paths
    def get_path_length(self,path):
        return self.each_path_legth[path]-1



    
    

In [9]:
# tm = spike_tm(6+1,2,250,10)
# each_t_user_pairs={0:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   1:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   2:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   3:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   4:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   5:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   6:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   7:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   8:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)],
#                   9:[(1,2),(1,3),(1,4),(1,5),(1,6),(1,7)]}
# each_t_each_request_demand = {}
# for time in range(10):
#     traffic = tm.at_time(time)
# #     for item in traffic:
# #         print('item',item)
#     printed_pairs = []
#     user_indx = 0
#     for i in range(6):
#         for j in range(6):
#             if i!=j:
                
#                 if (i,j) not in printed_pairs and (j,i) not in printed_pairs and user_indx <6:
#                     printed_pairs.append((i,j))
#                     printed_pairs.append((j,i))
# #                             print("num_of_pairs %s time %s traffic from %s to %s is %s and user_indx %s"%(num_of_pairs, time,i,j,traffic[i][j],user_indx))
#                     request = each_t_user_pairs[time][user_indx]
#                     user_indx+=1
#                     demand = max(1,traffic[i][j])
#                     print("for time %s node %s to node %s user_indx %s demand %s"%(time,i,j,user_indx,demand))
#                     try:
#                         each_t_each_request_demand[time][request] = demand
#                     except:
#                         each_t_each_request_demand[time] = {}
#                         each_t_each_request_demand[time][request] = demand

for time 0 node 0 to node 1 user_indx 1 demand 1
for time 0 node 0 to node 2 user_indx 2 demand 1
for time 0 node 0 to node 3 user_indx 3 demand 1
for time 0 node 0 to node 4 user_indx 4 demand 1
for time 0 node 0 to node 5 user_indx 5 demand 1
for time 0 node 1 to node 2 user_indx 6 demand 1
for time 1 node 0 to node 1 user_indx 1 demand 340.83907232208276
for time 1 node 0 to node 2 user_indx 2 demand 1
for time 1 node 0 to node 3 user_indx 3 demand 1
for time 1 node 0 to node 4 user_indx 4 demand 1
for time 1 node 0 to node 5 user_indx 5 demand 1
for time 1 node 1 to node 2 user_indx 6 demand 1
for time 2 node 0 to node 1 user_indx 1 demand 1
for time 2 node 0 to node 2 user_indx 2 demand 1
for time 2 node 0 to node 3 user_indx 3 demand 1
for time 2 node 0 to node 4 user_indx 4 demand 1
for time 2 node 0 to node 5 user_indx 5 demand 1
for time 2 node 1 to node 2 user_indx 6 demand 1
for time 3 node 0 to node 1 user_indx 1 demand 1
for time 3 node 0 to node 2 user_indx 2 demand 1
for